# Albums and Songs Lab

### Introduction

In this lesson, we'll use the skills we have learned over the past several lessons to answer questions about the top songs, artists and albums over the past fifty years.

### Working with Songs

Let's start by working with data regarding top 500 albums according to the Rolling Stone Magazine.

In [1]:
import pandas as pd
url = "https://raw.githubusercontent.com/data-eng-10-21/mod-1-a-data-structures/master/6-top-songs/data.csv"
df = pd.read_csv(url)
albums = df.to_dict('records')

In [3]:
albums[:2]

[{'number': 1,
  'year': 1967,
  'album': "Sgt. Pepper's Lonely Hearts Club Band",
  'artist': 'The Beatles',
  'genre': 'Rock',
  'subgenre': 'Rock & Roll, Psychedelic Rock'},
 {'number': 2,
  'year': 1966,
  'album': 'Pet Sounds',
  'artist': 'The Beach Boys',
  'genre': 'Rock',
  'subgenre': 'Pop Rock, Psychedelic Rock'}]

In [2]:
len(albums)

478

> Well, 478.

Let's write some functions to help us better explore the data.

* `all_albums` - Takes an argument of albums and returns the list of album names.

* `all_artists` - Takes argument of list of albums and returns a list of all artists (where each element is a string), and no artist is repeated.

* `find_by_name` - Has one argument of `album_name`. Returns a dictionary of the correct album, or `None` if no album is found.

* `find_by_ranks` - Takes `begin_rank` and `end_rank` as arguments.  Also possible to execute the function by just providing the `begin_rank` or `end_rank` (and not both).  If no arguments are provided the entire list of albums are returned.

* `find_by_years` - Takes `begin_year` and `end_year` as arguments, and returns a list of dictionaries for albums between those years.  Also possible to execute the function by just providing the `begin_year` or `end_year` (and not both).

In [4]:
def get_album_data_by_key(albums: list[dict], key_name: str):
  return [album[key_name] for album in albums]

In [6]:
def get_all_albums(albums: list[dict]):
  album_names = get_album_data_by_key(albums, 'album' )
  return album_names

# get_all_albums(albums)

In [13]:
def find_album_by_name(albums: list[dict], name: str) -> dict:
  for album in albums:
    if album['album'] == name:
      return album

In [15]:
find_album_by_name(albums, 'Elvis esley')

In [19]:
def find_album_by_ranks(albums: list[dict], begin_rank: float=float('-inf'), end_rank: float=float('inf')):
  return [album for album in albums if album['number'] > begin_rank and album['number'] < end_rank]


In [20]:
def find_album_by_year(albums: list[dict], begin_year: float=float('-inf'), end_year: float=float('inf')):
  return [album for album in albums if album['year'] > begin_year and album['year'] < end_year]


### Working with Songs

Next, let's load up data related to songs, and data that connects albums and songs.

In [21]:
import pandas as pd
songs_url = "https://raw.githubusercontent.com/data-eng-10-21/mod-1-a-data-structures/master/6-top-songs/top-500-songs.txt"
songs_df = pd.read_csv(songs_url, sep='\t', header = None, names = ['rank', 'song', 'artist', 'year'])
songs = songs_df.to_dict('records')

track_url = "https://raw.githubusercontent.com/data-eng-10-21/mod-1-a-data-structures/master/6-top-songs/track_data.json"
albums_and_tracks = pd.read_json(track_url)
albums_tracks = albums_and_tracks.to_dict('records')

In [22]:
songs[:2]

[{'rank': 1,
  'song': 'Like a Rolling Stone',
  'artist': 'Bob Dylan',
  'year': 1965},
 {'rank': 2,
  'song': 'Satisfaction',
  'artist': 'The Rolling Stones',
  'year': 1965}]

In [68]:
albums_tracks[0]


{'artist': 'The Beatles',
 'album': "Sgt. Pepper's Lonely Hearts Club Band",
 'tracks': ["Sgt. Pepper's Lonely Hearts Club Band - Remix",
  'With A Little Help From My Friends - Remix',
  'Lucy In The Sky With Diamonds - Remix',
  'Getting Better - Remix',
  'Fixing A Hole - Remix',
  "She's Leaving Home - Remix",
  'Being For The Benefit Of Mr. Kite! - Remix',
  'Within You Without You - Remix',
  "When I'm Sixty-Four - Remix",
  'Lovely Rita - Remix',
  'Good Morning Good Morning - Remix',
  "Sgt. Pepper's Lonely Hearts Club Band (Reprise) - Remix",
  'A Day In The Life - Remix',
  "Sgt. Pepper's Lonely Hearts Club Band - Take 9 And Speech",
  'With A Little Help From My Friends - Take 1 / False Start And Take 2 / Instrumental',
  'Lucy In The Sky With Diamonds - Take 1',
  'Getting Better - Take 1 / Instrumental And Speech At The End',
  'Fixing A Hole - Speech And Take 3',
  "She's Leaving Home - Take 1 / Instrumental",
  'Being For The Benefit Of Mr. Kite! - Take 4',
  'Within You

In [47]:
# first, we edit our data structure. Edit the album data structure so each song has an artist and album
def edit_albums_tracks(albums: list[dict]) -> list[dict]:
  clean_album_tracks = []
  for album in albums_tracks:
    filtered_tracks = [track.split(' - ')[0].split('(Reprise)')[0] for track in album['tracks']]
    clean_tracks = [{'artist': album['artist'], 'album': album['album'], 'name': \
                    track} for track in filtered_tracks]
    clean_album_tracks = clean_album_tracks + clean_tracks

  return clean_album_tracks

In [48]:
tracks_list = edit_albums_tracks(albums_tracks)
len(tracks_list)

7375

In [54]:
def remove_duplicate_tracks(track_list):
  seen_track_names = []
  seen_tracks = []
  for track in tracks_list:
    if track['name'] not in seen_track_names:
      seen_track_names.append(track['name'])
      seen_tracks.append(track)
  return seen_tracks

In [56]:
clean_tracks_list = remove_duplicate_tracks(tracks_list)
len(clean_tracks_list)

6402

In [76]:
def get_counter(albums_list):
  albums = list(set([track['album'] for track in albums_list]))
  album_names = {}
  for track in albums_tracks:
    album_names[track['album']] = 0
  return album_names


In [77]:
# find top albums
# create new data struct. for album names
album_names = get_counter(albums_tracks)

len(album_names)

475

In [92]:
def get_top_songs(songs_list, albums_tracks_list):
  tracks = edit_albums_tracks(albums_tracks_list)
  counter = get_counter(albums_tracks_list)
  clean_tracks = remove_duplicate_tracks(tracks)
  for song in songs_list:
    for track in clean_tracks:
      if track['name'] == song['song']:
          counter[track['album']] += 1
  return sorted(counter.items(), key = lambda name: name[1], reverse=True)




In [93]:
top_album = get_top_songs(songs, albums_tracks)
top_album[:10]

[('Elvis Presley', 6),
 ('The Great Twenty Eight', 5),
 ('Are You Experienced', 4),
 ('Bringing It All Back Home', 4),
 ('Pet Sounds', 3),
 ('Highway 61 Revisited', 3),
 ('Abbey Road', 3),
 ('Otis Blue: Otis Redding Sings Soul', 3),
 ('At Folsom Prison', 3),
 ("Sgt. Pepper's Lonely Hearts Club Band", 2)]

In [94]:
def get_album_artist(albums_list):
  album_artist = {}
  for album in albums_list:
    for track_album in albums_tracks:
      if album[0] == track_album['album']:
        print(track_album['artist'])


In [95]:
get_album_artist(top_album[:10])

Elvis Presley
Chuck Berry
The Jimi Hendrix Experience
Bob Dylan
The Beach Boys
Bob Dylan
The Beatles
Otis Redding
Johnny Cash
The Beatles


* Write functions that perform the following:

`album_most_top_songs` -
    * Returns the name of the artist and album that has that most songs featured on the top 500 songs list

`top_ten_albums_by_songs` - returns a dictionary with the 10 albums that have the most songs that appear in the top songs list. The album names should be the keys and the corresponding values should be the number of songs that appear on the top 500 list.